In [19]:
import numpy as np

def cholesky_decomposition(A):
    """
    Performs Cholesky decomposition of a symmetric, positive-definite matrix A.
    A = L * L.T, where L is a lower triangular matrix.
    
    Parameters:
    A : 2D numpy array (n x n) - Input matrix (must be symmetric and positive definite).
    
    Returns:
    L : 2D numpy array (n x n) - Lower triangular matrix such that A = L * L.T.
    """
    # Get the size of the matrix (number of rows/columns, since it's square)
    n = A.shape[0]
    
    # Initialize an empty lower triangular matrix L with the same shape as A
    L = np.zeros_like(A, dtype=np.float64)  # Ensure the matrix is in float64 for numerical stability

    print("Initial Matrix A:")
    print(A)
    
    # Loop over each row i
    for i in range(n):
        # Loop over each column j up to row i (since it's a lower triangular matrix, j <= i)
        for j in range(i + 1):
            if i == j:
                # Diagonal element calculation for L[i, i]
                # L[i, i] = sqrt(A[i, i] - sum(L[i, :i]^2))
                diag_sum = np.sum(L[i, :i]**2)  # This is the sum of squares of the elements in the current row up to column i
                L[i, j] = np.sqrt(A[i, i] - diag_sum)
                
                # Print detailed information for diagonal element computation
                print(f"\nStep ({i+1},{j+1}): Diagonal element L[{i},{j}]")
                print(f"L[{i},{j}] = sqrt(A[{i},{i}] - sum(L[{i},:i]^2))")
                print(f"L[{i},{j}] = sqrt({A[i,i]:.6f} - {diag_sum:.6f}) = {L[i,j]:.6f}")
                print(f"Elements contributing to sum(L[{i},:i]^2): {L[i,:i]**2}")
                
            else:
                # Off-diagonal element calculation for L[i, j]
                # L[i, j] = (A[i, j] - sum(L[i, :j] * L[j, :j])) / L[j, j]
                off_diag_sum = np.sum(L[i, :j] * L[j, :j])  # This is the sum of products of the corresponding elements of row i and column j
                L[i, j] = (A[i, j] - off_diag_sum) / L[j, j]
                
                # Print detailed information for off-diagonal element computation
                print(f"\nStep ({i+1},{j+1}): Off-diagonal element L[{i},{j}]")
                print(f"L[{i},{j}] = (A[{i},{j}] - sum(L[{i},:j] * L[{j},:j])) / L[{j},{j}]")
                print(f"L[{i},{j}] = ({A[i,j]:.6f} - {off_diag_sum:.6f}) / {L[j,j]:.6f} = {L[i,j]:.6f}")
                print(f"Elements contributing to sum(L[{i},:j] * L[{j},:j]): {L[i,:j] * L[j,:j]}")

        # Display the current state of the matrix L after processing each row i
        print(f"\nL matrix after processing row {i+1}:\n{L}")

    # Print the final L matrix
    print("\nFinal L matrix:\n", L)
    
    return L

# Example usage
A = np.array([[9, 3, 3],
              [3, 7, 1],
              [3, 1, 5]], dtype=np.float64)

L = cholesky_decomposition(A)


Initial Matrix A:
[[9. 3. 3.]
 [3. 7. 1.]
 [3. 1. 5.]]

Step (1,1): Diagonal element L[0,0]
L[0,0] = sqrt(A[0,0] - sum(L[0,:i]^2))
L[0,0] = sqrt(9.000000 - 0.000000) = 3.000000
Elements contributing to sum(L[0,:i]^2): []

L matrix after processing row 1:
[[3. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Step (2,1): Off-diagonal element L[1,0]
L[1,0] = (A[1,0] - sum(L[1,:j] * L[0,:j])) / L[0,0]
L[1,0] = (3.000000 - 0.000000) / 3.000000 = 1.000000
Elements contributing to sum(L[1,:j] * L[0,:j]): []

Step (2,2): Diagonal element L[1,1]
L[1,1] = sqrt(A[1,1] - sum(L[1,:i]^2))
L[1,1] = sqrt(7.000000 - 1.000000) = 2.449490
Elements contributing to sum(L[1,:i]^2): [1.]

L matrix after processing row 2:
[[3.         0.         0.        ]
 [1.         2.44948974 0.        ]
 [0.         0.         0.        ]]

Step (3,1): Off-diagonal element L[2,0]
L[2,0] = (A[2,0] - sum(L[2,:j] * L[0,:j])) / L[0,0]
L[2,0] = (3.000000 - 0.000000) / 3.000000 = 1.000000
Elements contributing to sum(L[2,:j] * L[0,:j]): []



In [20]:
import numpy as np

def ldl_decomposition(A):
    """
    Performs LDL decomposition of a symmetric matrix A.
    A = L * D * L.T, where L is a lower triangular matrix with ones on the diagonal,
    and D is a diagonal matrix.
    
    Parameters:
    A : 2D numpy array (n x n) - Input matrix (must be symmetric).
    
    Returns:
    L : 2D numpy array (n x n) - Lower triangular matrix with ones on diagonal.
    D : 1D numpy array (n) - Diagonal elements of D matrix.
    """
    n = A.shape[0]
    L = np.eye(n)
    D = np.zeros(n)

    print("Initial Matrix A:")
    print(A)
    
    for j in range(n):
        # Step: Compute D[j]
        D[j] = A[j, j] - np.sum(L[j, :j]**2 * D[:j])
        print(f"\nStep {j+1}: Diagonal element D[{j}] = A[{j},{j}] - sum(L[{j},:j]^2 * D[:j]) = {D[j]:.6f}")
        
        for i in range(j + 1, n):
            # Step: Compute L[i, j]
            L[i, j] = (A[i, j] - np.sum(L[i, :j] * L[j, :j] * D[:j])) / D[j]
            print(f"Step {j+1}.{i+1}: L[{i},{j}] = (A[{i},{j}] - sum(L[{i},:j] * L[{j},:j] * D[:j])) / D[{j}] = {L[i, j]:.6f}")
        
        # Display L and D matrices after each step
        print(f"L matrix after processing column {j+1}:\n{L}")
        print(f"D vector after processing column {j+1}:\n{D}")

    print("\nFinal L matrix:\n", L)
    print("Final D vector:\n", D)

    return L, D

# Example usage
A = np.array([[4, 12, -16],
              [12, 37, -43],
              [-16, -43, 98]])

L, D = ldl_decomposition(A)


Initial Matrix A:
[[  4  12 -16]
 [ 12  37 -43]
 [-16 -43  98]]

Step 1: Diagonal element D[0] = A[0,0] - sum(L[0,:j]^2 * D[:j]) = 4.000000
Step 1.2: L[1,0] = (A[1,0] - sum(L[1,:j] * L[0,:j] * D[:j])) / D[0] = 3.000000
Step 1.3: L[2,0] = (A[2,0] - sum(L[2,:j] * L[0,:j] * D[:j])) / D[0] = -4.000000
L matrix after processing column 1:
[[ 1.  0.  0.]
 [ 3.  1.  0.]
 [-4.  0.  1.]]
D vector after processing column 1:
[4. 0. 0.]

Step 2: Diagonal element D[1] = A[1,1] - sum(L[1,:j]^2 * D[:j]) = 1.000000
Step 2.3: L[2,1] = (A[2,1] - sum(L[2,:j] * L[1,:j] * D[:j])) / D[1] = 5.000000
L matrix after processing column 2:
[[ 1.  0.  0.]
 [ 3.  1.  0.]
 [-4.  5.  1.]]
D vector after processing column 2:
[4. 1. 0.]

Step 3: Diagonal element D[2] = A[2,2] - sum(L[2,:j]^2 * D[:j]) = 9.000000
L matrix after processing column 3:
[[ 1.  0.  0.]
 [ 3.  1.  0.]
 [-4.  5.  1.]]
D vector after processing column 3:
[4. 1. 9.]

Final L matrix:
 [[ 1.  0.  0.]
 [ 3.  1.  0.]
 [-4.  5.  1.]]
Final D vector:
 [4

In [21]:
def cholesky_rank1_update(L, u):
    """
    Performs a rank-1 update to the Cholesky factor L, with detailed step-by-step logging.
    Updates the matrix such that the new factor corresponds to A + u * u.T.

    Parameters:
    L : 2D numpy array (n x n) - Lower triangular matrix from the Cholesky decomposition of A.
    u : 1D numpy array (n) - Vector used in the rank-1 update.
    
    Returns:
    L_updated : 2D numpy array (n x n) - Updated lower triangular matrix.
    """
    n = L.shape[0]
    u = u.copy()  # Create a copy of the update vector

    L = L.astype(np.float64)
    u = u.astype(np.float64)

    print("\nInitial L matrix:\n", L)
    print("Initial update vector u:\n", u)

    for i in range(n):
        r = np.sqrt(L[i, i]**2 + u[i]**2)
        c = r / L[i, i]
        s = u[i] / L[i, i]
        print(f"\nStep {i+1}:")
        print(f"Updating diagonal element L[{i},{i}]: sqrt(L[{i},{i}]^2 + u[{i}]^2) = {r:.6f}")
        L[i, i] = r
        
        if i < n - 1:
            print(f"Updating off-diagonal elements for row {i+1}:")
            for j in range(i+1, n):
                print(f"  L[{j},{i}] = (L[{j},{i}] + s * u[{j}]) / c = ({L[j, i]} + {s} * {u[j]}) / {c}")
            
            # Perform vectorized updates after printing details
            L[i+1:, i] = (L[i+1:, i] + s * u[i+1:]) / c
            u[i+1:] = c * u[i+1:] - s * L[i+1:, i]
            print(f"Updated vector u after step {i+1}:\n", u)
        
        print(f"L matrix after step {i+1}:\n", L)

    print("\nFinal updated L matrix:\n", L)
    return L


In [22]:
# Example usage
A = np.array([[4, 2],
              [2, 3]], dtype=np.float64)

L = cholesky_decomposition(A)

# Vector u for the rank-1 update
u = np.array([1, 1], dtype=np.float64)

print('-----------------------------------')
print('Rank-1 update')
# Perform rank-1 update
L_updated = cholesky_rank1_update(L, u)


Initial Matrix A:
[[4. 2.]
 [2. 3.]]

Step (1,1): Diagonal element L[0,0]
L[0,0] = sqrt(A[0,0] - sum(L[0,:i]^2))
L[0,0] = sqrt(4.000000 - 0.000000) = 2.000000
Elements contributing to sum(L[0,:i]^2): []

L matrix after processing row 1:
[[2. 0.]
 [0. 0.]]

Step (2,1): Off-diagonal element L[1,0]
L[1,0] = (A[1,0] - sum(L[1,:j] * L[0,:j])) / L[0,0]
L[1,0] = (2.000000 - 0.000000) / 2.000000 = 1.000000
Elements contributing to sum(L[1,:j] * L[0,:j]): []

Step (2,2): Diagonal element L[1,1]
L[1,1] = sqrt(A[1,1] - sum(L[1,:i]^2))
L[1,1] = sqrt(3.000000 - 1.000000) = 1.414214
Elements contributing to sum(L[1,:i]^2): [1.]

L matrix after processing row 2:
[[2.         0.        ]
 [1.         1.41421356]]

Final L matrix:
 [[2.         0.        ]
 [1.         1.41421356]]
-----------------------------------
Rank-1 update

Initial L matrix:
 [[2.         0.        ]
 [1.         1.41421356]]
Initial update vector u:
 [1. 1.]

Step 1:
Updating diagonal element L[0,0]: sqrt(L[0,0]^2 + u[0]^2) = 